# Data Exploration

In [1]:
import pandas as pd
import numpy as np

In [2]:
team_names = pd.read_csv("./data/WDataFiles/WTeams.csv")
seasons = pd.read_csv("./data/WDataFiles/WSeasons.csv")
seeds = pd.read_csv("./data/WDataFiles/WNCAATourneySeeds.csv")
season_results = pd.read_csv("./data/WDataFiles/WRegularSeasonCompactResults.csv")
tourney_results = pd.read_csv("./data/WDataFiles/WNCAATourneyCompactResults.csv")
cities = pd.read_csv("./data/WDataFiles/WCities.csv")
game_cities = pd.read_csv("./data/WDataFiles/WGameCities.csv")

In [3]:
seasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,10/27/1997,East,Midwest,Mideast,West
1,1999,10/26/1998,East,Mideast,Midwest,West
2,2000,11/1/1999,East,Midwest,Mideast,West
3,2001,10/30/2000,East,Midwest,Mideast,West
4,2002,10/29/2001,East,West,Mideast,Midwest


In [4]:
seeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [5]:
season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


### Intuitions to follow up on

* Does being on home court increase probability of a win?
* Does the outcome of the prior game for this team give them "momentum"?
* Do Win/Loss ratios for the season so far correlate with wins?
* Does seed distance from opponent correlate with winning?
* Does prior season normalized points scored with respect to total distribution correlate with winning?
* same question for season-to-date
* same question for historical and season-to-date defense
* can we easily determine distances from all cities to all other cities?  Do we know the home city for each team?  If so can we use distance from home city to game city as a proxy for travel fatigue? And does it affect winning outcomes?
* are relative win-ratios from regular season play predictive of tournament wins?
* are there different regional strengths?

## Stage 1 Predictions

The first portion of this exercise involves successfully predicting matchups from past tournaments.  The goal is to take the 64 teams in each tournament, and predict each possible matchup.  To get the formatting right, I'll build a model that extracts the seed for each tournament, and tries to learn a naive predictor just based on seed difference between teams.  Steps:

* 1: Make training set from tournament results.
  - extract seed 1 and seed 2
  - make Y vector, 1 for a team 1 victory, 0 for team 2 victory
  - teams in a match ordered by numeric ID

* 2: train model to convergence on entire batch

* 3: Make a test set from seeds list for all possible combinations and seed numbers

* 4: predict results and confirm an output size of 8,064 data rows.


In [32]:
from sklearn.linear_model import LogisticRegression
import itertools
import csv
    
def figure_team_1(row):
    if row['Team1Win']:
        return row['WTeamID']
    else:
        return row['LTeamID']

def figure_team_2(row):
    if row['Team1Win']:
        return row['LTeamID']
    else:
        return row['WTeamID']

def training_set_baseline(tourney_results_df, seeds_df):
    tourney_df = tourney_results_df[['Season','WTeamID','LTeamID']]
    tourney_df['Team1Win'] = tourney_df['WTeamID'] < tourney_df['LTeamID']
    tourney_df['Team1'] = tourney_df.apply(figure_team_1,axis=1)
    tourney_df['Team2'] = tourney_df.apply(figure_team_2,axis=1)
    training_df = pd.merge(tourney_df, seeds_df, left_on=['Season', 'Team1'], right_on=['Season', 'TeamID'])
    training_df = training_df.rename(index=str, columns={"Seed": "Team1Seed"})
    training_df = training_df.drop(['TeamID'], axis=1)
    training_df = pd.merge(training_df, seeds_df, left_on=['Season', 'Team2'], right_on=['Season', 'TeamID'])
    training_df = training_df.rename(index=str, columns={"Seed": "Team2Seed"})
    training_df = training_df.drop(['TeamID'], axis=1)
    training_df['SeedNum1'] = training_df.apply(lambda row: int(row['Team1Seed'][1:3]), axis=1)
    training_df['SeedNum2'] = training_df.apply(lambda row: int(row['Team2Seed'][1:3]), axis=1)
    training_df['SeedDelta'] = (training_df['SeedNum1'] - training_df['SeedNum2'])
    seed_mu = training_df['SeedDelta'].mean()
    seed_sigma = training_df['SeedDelta'].std()
    training_df['NormalizedSeedDelta'] = (training_df['SeedDelta'] - seed_mu) / seed_sigma
    return training_df, { 'seed_mu': seed_mu, 'seed_sigma': seed_sigma }

def test_set_baseline(seeds_df, training_meta):
    test_seeds = seeds_df[['Season', 'Seed', 'TeamID']]
    test_seeds['SeedNum'] = test_seeds.apply(lambda row: int(row['Seed'][1:3]), axis=1)
    seasons = np.unique(test_seeds.Season.values)
    test_set = [] # output_key (SSSS_XXXX_YYYY), seed1, seed2, seed_delta
    seasons = [ss for ss in seasons if ss >= 2014 and ss < 2018]
    for s in seasons:
        seed_set = test_seeds[test_seeds.Season == s].values
        for seed_pair in itertools.combinations(seed_set, 2):
            id_a = seed_pair[0][2]
            id_b = seed_pair[1][2]
            t1 = 0
            t2 = 0
            s1 = 0
            s2 = 0
            if id_a < id_b:
                t1 = id_a
                t2 = id_b
                s1 = seed_pair[0][3]
                s2 = seed_pair[1][3]
            else:
                t1 = id_b
                t2 = id_a
                s1 = seed_pair[1][3]
                s2 = seed_pair[0][3]
            seed_delta = ((s1 - s2) - training_meta['seed_mu']) / training_meta['seed_sigma']
            output_key = "%s_%s_%s" % (s, t1, t2)
            test_set.append([output_key, s1, s2, seed_delta])
    test_set = np.array(test_set)
    return test_set

def predict_with_naive_model(test_ds, model):
    test_x = test_ds[:, 3].astype(float)
    test_x = test_x.reshape(len(test_x), 1)
    predictions = model.predict_proba(test_x)[:, 1]
    return predictions
    
def build_naive_model(training_df):
    x_train = training_df['NormalizedSeedDelta'].values
    y_train = np.vectorize(lambda x: 1 if x else 0)(training_df['Team1Win'].values)
    x_train = x_train.reshape(len(x_train), 1)
    y_train = y_train.reshape(len(y_train), )
    naive_model = LogisticRegression()
    naive_model = naive_model.fit(x_train, y_train)
    print("SCORE: ", naive_model.score(x_train, y_train))
    return naive_model

def write_predictions(test_set, preds):
    with open("./data/outputs/stage_1_predictions.csv", "w") as f:
        wtr = csv.writer(f)
        wtr.writerow(['ID','Pred'])
        for i in range(len(test_set)):
            out_key = test_set[i][0]
            prediction = preds[i]
            wtr.writerow([out_key,prediction])
    

In [27]:
training_df, training_meta = training_set_baseline(tourney_results, seeds)

/Users/ethan/anaconda/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ethan/anaconda/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ethan/anaconda/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [28]:
training_df.head(10)

,Season,WTeamID,LTeamID,Team1Win,Team1,Team2,Team1Seed,Team2Seed,SeedNum1,SeedNum2,SeedDelta,NormalizedSeedDelta
0,1998,3104,3422,True,3104,3422,X02,X15,2,15,-13,-1.751175
1,1998,3104,3417,True,3104,3417,X02,X07,2,7,-5,-0.672212
2,1998,3417,3276,False,3276,3417,X10,X07,10,7,3,0.406752
3,1998,3256,3104,False,3104,3256,X02,X03,2,3,-1,-0.132730
4,1998,3256,3155,False,3155,3256,X06,X03,6,3,3,0.406752
5,1998,3256,3221,False,3221,3256,X14,X03,14,3,11,1.485716
6,1998,3112,3365,True,3112,3365,W03,W14,3,14,-11,-1.481434
7,1998,3112,3438,True,3112,3438,W03,W06,3,6,-3,-0.402471
8,1998,3438,3374,False,3374,3438,W11,W06,11,6,5,0.676493
9,1998,3163,3112,False,3112,3163,W03,W02,3,2,1,0.137011


In [29]:
current_model = build_naive_model(training_df)
current_test_set = test_set_baseline(seeds, training_meta)
predictions = predict_with_naive_model(current_test_set, current_model)

SCORE:  0.778571428571


## write first naive predictions file to get the format down right

In [33]:
write_predictions(current_test_set, predictions)        

## RESULTS SO FAR:
* ACCURACY ON TRAINING: 0.7786
* TESTING SCORE: 0.4626

## including other features

Some other features that are likely to be helpful are offensive and defensive information for each team, so lets' try median offensive and defensive points for the regular season as a feature.

In [90]:
def build_all_scores(season_df):
    w_scores = season_df[['Season', 'WTeamID', 'WScore']].rename(index=str, columns={"WTeamID": "TeamID", "WScore": "Score"})
    l_scores = season_df[['Season', 'LTeamID', 'LScore']].rename(index=str, columns={"LTeamID": "TeamID", "LScore": "Score"})
    all_scores = pd.concat([w_scores, l_scores])
    grouped_scores = all_scores.groupby(['Season', 'TeamID'], as_index=False).aggregate(np.median)
    return grouped_scores
    
def training_set_with_scores(tourney_results_df, seeds_df, season_df):
    baseline_df, training_meta = training_set_baseline(tourney_results_df, seeds_df)
    grouped_scores = build_all_scores(season_df)
    score_mu = grouped_scores['Score'].mean()
    score_sigma = grouped_scores['Score'].std()
    training_meta['score_mu'] = score_mu
    training_meta['score_sigma'] = score_sigma

    training_df = pd.merge(baseline_df, grouped_scores, left_on=['Season', 'Team1'], right_on=['Season', 'TeamID'])
    training_df = training_df.rename(index=str, columns={"Score": "Team1MedianScore"})
    training_df = training_df.drop(['TeamID'], axis=1)
    training_df = pd.merge(training_df, grouped_scores, left_on=['Season', 'Team2'], right_on=['Season', 'TeamID'])
    training_df = training_df.rename(index=str, columns={"Score": "Team2MedianScore"})
    training_df = training_df.drop(['TeamID'], axis=1)
    training_df['Team1NormedScore'] = (training_df['Team1MedianScore'] - score_mu) / (score_sigma ** 2)
    training_df['Team2NormedScore'] = (training_df['Team2MedianScore'] - score_mu) / (score_sigma ** 2)
    return training_df, training_meta

def build_multifeature_naive_model(training_df):
    x_train = training_df[['NormalizedSeedDelta', 'Team1NormedScore', 'Team2NormedScore']].values
    y_train = np.vectorize(lambda x: 1 if x else 0)(training_df['Team1Win'].values)
    x_train = x_train.reshape(len(x_train), 3)
    y_train = y_train.reshape(len(y_train), )
    naive_model = LogisticRegression()
    naive_model = naive_model.fit(x_train, y_train)
    print("SCORE: ", naive_model.score(x_train, y_train))
    return naive_model

def test_set_with_scores(seeds_df, season_df, training_meta):
    test_seeds = seeds_df[['Season', 'Seed', 'TeamID']]
    test_seeds['SeedNum'] = test_seeds.apply(lambda row: int(row['Seed'][1:3]), axis=1)
    seasons = np.unique(test_seeds.Season.values)
    grouped_scores = build_all_scores(season_df)
    test_set = [] # output_key (SSSS_XXXX_YYYY), seed1, seed2, seed_delta, norm_score_1, norm_score_2
    seasons = [ss for ss in seasons if ss >= 2014 and ss < 2018]
    for s in seasons:
        seed_set = test_seeds[test_seeds.Season == s].values
        for seed_pair in itertools.combinations(seed_set, 2):
            id_a = seed_pair[0][2]
            id_b = seed_pair[1][2]
            t1 = 0
            t2 = 0
            s1 = 0
            s2 = 0
            if id_a < id_b:
                t1 = id_a
                t2 = id_b
                s1 = seed_pair[0][3]
                s2 = seed_pair[1][3]
            else:
                t1 = id_b
                t2 = id_a
                s1 = seed_pair[1][3]
                s2 = seed_pair[0][3]
            seed_delta = ((s1 - s2) - training_meta['seed_mu']) / training_meta['seed_sigma']
            output_key = "%s_%s_%s" % (s, t1, t2)
            t1_norm_score = 
            test_set.append([output_key, s1, s2, seed_delta])
    test_set = np.array(test_set)
    return test_set

In [86]:
training_set, meta = training_set_with_scores(tourney_results, seeds,season_results)

/Users/ethan/anaconda/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ethan/anaconda/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ethan/anaconda/envs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [91]:
training_set.head()

,Season,WTeamID,LTeamID,Team1Win,Team1,Team2,Team1Seed,Team2Seed,SeedNum1,SeedNum2,SeedDelta,NormalizedSeedDelta,Team1MedianScore,Team2MedianScore,Team1NormedScore,Team2NormedScore
0,1998,3104,3422,True,3104,3422,X02,X15,2,15,-13,-1.751175,74.0,75.0,0.197045,0.217213
1,1998,3104,3417,True,3104,3417,X02,X07,2,7,-5,-0.672212,74.0,80.0,0.197045,0.318050
2,1998,3417,3276,False,3276,3417,X10,X07,10,7,3,0.406752,77.0,80.0,0.257547,0.318050
3,1998,3256,3104,False,3104,3256,X02,X03,2,3,-1,-0.132730,74.0,86.0,0.197045,0.439054
4,1998,3256,3155,False,3155,3256,X06,X03,6,3,3,0.406752,73.0,86.0,0.176878,0.439054


In [93]:
multifeature_model = build_multifeature_naive_model(training_set)

SCORE:  0.786507936508


In [53]:
current_test_set = test_set_with_scores(seeds, season_results, training_meta)
predictions = predict_with_naive_model(current_test_set, current_model)

In [66]:
grouped_scores = all_scores.groupby(['Season', 'TeamID'], as_index=False).aggregate(np.median)

In [68]:
grouped_scores.head()

,Season,TeamID,Score
0,1998,3102,58.5
1,1998,3103,68.0
2,1998,3104,74.0
3,1998,3106,63.0
4,1998,3108,68.0


In [69]:
training_set.head()

,Season,WTeamID,LTeamID,Team1Win,Team1,Team2,Team1Seed,Team2Seed,SeedNum1,SeedNum2,SeedDelta,NormalizedSeedDelta
0,1998,3104,3422,True,3104,3422,X02,X15,2,15,-13,-1.751175
1,1998,3104,3417,True,3104,3417,X02,X07,2,7,-5,-0.672212
2,1998,3417,3276,False,3276,3417,X10,X07,10,7,3,0.406752
3,1998,3256,3104,False,3104,3256,X02,X03,2,3,-1,-0.132730
4,1998,3256,3155,False,3155,3256,X06,X03,6,3,3,0.406752


In [76]:
training_df = pd.merge(training_set, grouped_scores, left_on=['Season', 'Team1'], right_on=['Season', 'TeamID'])
training_df = training_df.rename(index=str, columns={"Score": "Team1MedianScore"})
training_df = training_df.drop(['TeamID'], axis=1)
training_df = pd.merge(training_df, grouped_scores, left_on=['Season', 'Team2'], right_on=['Season', 'TeamID'])
training_df = training_df.rename(index=str, columns={"Score": "Team2MedianScore"})
training_df = training_df.drop(['TeamID'], axis=1)

In [78]:
training_df.head(20)

,Season,WTeamID,LTeamID,Team1Win,Team1,Team2,Team1Seed,Team2Seed,SeedNum1,SeedNum2,SeedDelta,NormalizedSeedDelta,Team1MedianScore,Team2MedianScore
0,1998,3104,3422,True,3104,3422,X02,X15,2,15,-13,-1.751175,74.0,75.0
1,1998,3104,3417,True,3104,3417,X02,X07,2,7,-5,-0.672212,74.0,80.0
2,1998,3417,3276,False,3276,3417,X10,X07,10,7,3,0.406752,77.0,80.0
3,1998,3256,3104,False,3104,3256,X02,X03,2,3,-1,-0.132730,74.0,86.0
4,1998,3256,3155,False,3155,3256,X06,X03,6,3,3,0.406752,73.0,86.0
5,1998,3256,3221,False,3221,3256,X14,X03,14,3,11,1.485716,80.0,86.0
6,1998,3155,3274,True,3155,3274,X06,X11,6,11,-5,-0.672212,73.0,73.0
7,1998,3112,3365,True,3112,3365,W03,W14,3,14,-11,-1.481434,81.0,75.0
8,1998,3112,3438,True,3112,3438,W03,W06,3,6,-3,-0.402471,81.0,66.0
9,1998,3438,3374,False,3374,3438,W11,W06,11,6,5,0.676493,70.5,66.0


In [ ]:
training_df = training_df.rename(index=str, columns={"Score": "Team1MedianScore"})
training_df = training_df.drop(['TeamID'], axis=1)